In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\src\preprocessing.py').resolve().parent.parent))

In [22]:
nonzero3darray_folder_path = r'C:\Users\parkm\Desktop\miccai-stroke-2022\reference_repo\ATLAS2022-UNETR\data\NonZero3darray\imagesTr'

In [3]:
import os
import glob
import numpy as np
import nibabel as nib 

In [23]:
def recursive_find_all_files(top_folder_path:str, file_format:str) -> list:
    gathered_file_pathes = []

    for (root, directories, files) in os.walk(top_folder_path):
        for file in files:
            if file_format in file:
                file_path = os.path.join(root, file)
                gathered_file_pathes.append(file_path)

    return gathered_file_pathes

In [24]:
nonzero3darray_file_list = recursive_find_all_files(nonzero3darray_folder_path, '.nii.gz')

In [6]:
x_min_list, x_max_list, y_min_list, y_max_list, z_min_list, z_max_list = [],[],[],[],[],[]

### 1. 전체 데이터셋의 nozero 영역 확인해서 각 축별 최대,최소값 찾기

In [7]:
def get_nonzero3darray_shape(nifti_file_path: str) -> int:
    
    file_name = nifti_file_path.split('\\')[-1].rstrip('nii.gz')
    
    nii = nib.load(nifti_file_path)
    nii_array = nii.get_fdata()
    original_affine = nii.affine
    
    non_zeros = np.nonzero(nii_array>2)

    x_min = non_zeros[0].min()
    x_max = non_zeros[0].max()

    y_min = non_zeros[1].min()
    y_max = non_zeros[1].max()

    z_min = non_zeros[2].min()
    z_max = non_zeros[2].max()
    
    
    return x_min, x_max, y_min, y_max, z_min, z_max


In [8]:
for nonzero3darray_file in nonzero3darray_file_list:
    x_min, x_max, y_min, y_max, z_min, z_max = get_nonzero3darray_shape(nonzero3darray_file)

    x_min_list.append(x_min) 
    x_max_list.append(x_max)
    y_min_list.append(y_min) 
    y_max_list.append(y_max)
    z_min_list.append(z_min) 
    z_max_list.append(z_max) 


In [14]:
min(x_min_list)

20

In [13]:
max(x_max_list)

176

In [16]:
min(y_min_list)

18

In [17]:
max(y_max_list)

212

In [18]:
min(z_min_list)

0

In [19]:
max(z_max_list)

167

### 2. 코호트의 nonzero 영역을 특정하고, 2를 마진으로 줘서, 전체 데이터셋 크롭 (원본 nifti)

In [20]:
def get_hardcrop(nifti_file_path, save_path, x_min, x_max, y_min, y_max, z_min, z_max):
    file_name = nifti_file_path.split('\\')[-1].rstrip('nii.gz')
    
    nii = nib.load(nifti_file_path)
    nii_array = nii.get_fdata()
    nii_array = nii_array[x_min:x_max, y_min:y_max, z_min:z_max]
    original_affine = nii.affine
        
        
    img = nib.Nifti1Image(nii_array, original_affine) 
    img.to_filename(f'{save_path}/{file_name}.nii.gz')

    return None

In [25]:
for nonzero3darray_file in nonzero3darray_file_list:
    
    get_hardcrop(nonzero3darray_file, nonzero3darray_folder_path, 18, 178, 16, 214, 0, 169)

In [ ]:
## 잘 크롭됐는지 확인해보기

In [31]:
nii = nib.load(nonzero3darray_file_list[0])
nii_array = nii.get_fdata()

In [32]:
nii_array.shape

(160, 198, 169)

In [33]:
## 정상적으로 잘 크롭되었음 (178-18, 214-16, 169-0)

### 3. 코호트의 nonzero 영역을 특정하고, 2를 마진으로 줘서, 전체 데이터셋 크롭 (mask nifti)

In [39]:
nonzero3darray_folder_path = r'C:\Users\parkm\Desktop\miccai-stroke-2022\reference_repo\ATLAS2022-UNETR\data\NonZero3darray\labelsTr'

In [40]:
nonzero3darray_file_list = recursive_find_all_files(nonzero3darray_folder_path, '.nii.gz')

In [41]:
for nonzero3darray_file in nonzero3darray_file_list:
    
    get_hardcrop(nonzero3darray_file, nonzero3darray_folder_path, 18, 178, 16, 214, 0, 169)